## CLASS MODULE

In [7]:
import os
import sys
import os
import logging
from easydict import EasyDict
import numpy as np
import random
import time
import datetime
from deap import tools
from collections import OrderedDict
from pprint import pprint
import json

sys.path.insert(0, '../')
from utils_kyy.utils_graph import make_random_graph
from utils_kyy.create_toolbox import create_toolbox_for_NSGA_RWNN


class rwns_train:
    def __init__(self, json_file):
        self.root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        #self.root = os.getcwd()
        self.param_dir = os.path.join(self.root + '/parameters/', json_file)
        self.log_dir = os.path.join(self.root, 'log')

        f = open(self.param_dir)
        params = json.load(f)

        pprint(params)

        self.name = params['NAME']

        ## toolbox params
        self.args_train = EasyDict(params['ARGS_TRAIN'])
        self.data_path = params['DATA_PATH']
        self.run_code = params['RUN_CODE']
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '/'
        self.num_graph = params['NUM_GRAPH']
        self.toolbox = None

        ## GA params
        self.pop_size = params['POP_SIZE']
        self.ngen = params['NGEN']
        self.cxpb = params['CXPB']
        self.mutpb = params['MUTPB']

        ## logs
        log = OrderedDict()

        log['hp'] = self.args_train
        self.log = log
        self.train_log = None

    ## tool box and make graph
    def create_toolbox(self):
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '_' + self.name + '/'

        if not os.path.exists(self.stage_pool_path): os.makedirs(self.stage_pool_path)
        if not os.path.isdir(self.log_path): os.makedirs(self.log_path)
        self.log_file_name = self.log_path + 'logging.log'
        self.train_log_file_name = self.log_path + 'train_logging.log'

        logging.basicConfig(filename=self.log_file_name, level=logging.INFO)
        logging.info('Start to write log.')

        # num_graph = 100
        make_random_graph(self.num_graph, self.stage_pool_path)

        return create_toolbox_for_NSGA_RWNN(self.num_graph, self.args_train, self.stage_pool_path, self.data_path,
                                            self.log_file_name)

    ## Train

    def train(self):

        ## Parameters
        self.create_toolbox()
        POP_SIZE = self.pop_size

        ## train log
        train_log = OrderedDict()

        if self.toolbox is None:
            self.toolbox = self.create_toolbox()

        toolbox = self.toolbox

        # log에 기록할 stats
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "min", "max", "evals_time", "gen_time"

        # population 생성.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialion starts ...")
        logging.info("Initialion starts at " + now_str)
        init_start_time = time.time()

        pop = toolbox.population(n=POP_SIZE)

        ## fitness list
        fit_list = []

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)  # .evaluate는 tuple을 반환. 따라서 fitnesses는 튜플을 원소로 가지는 list
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit  # ind.fitness.values = (val_accuracy, flops) 튜플
            fit_list.append(fit)

        ## log 기록
        train_log[0] = fit_list
        self.train_log = train_log
        print(train_log)

        # This is just to assign the crowding distance to the individuals
        # no actual selection is done
        pop = toolbox.select(pop, len(pop))

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(invalid_ind), **record)
        print(logbook.stream)

        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialization is finished at", now_str)
        logging.info("Initialion is finished at " + now_str)

        init_time = time.time() - init_start_time
        logging.info("Initialion time = " + str(init_time) + "s")

        print()

        # Begin the generational process
        for gen in range(1, self.ngen):
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d %H:%M:%S')
            print("#####", gen, "th generation starts at", now_str)
            logging.info(str(gen) + "th generation starts at" + now_str)

            start_gen = time.time()
            # Vary the population
            offspring = tools.selTournamentDCD(pop, len(pop))
            offspring = [toolbox.clone(ind) for ind in offspring]

            for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
                if random.random() <= self.cxpb:
                    toolbox.mate(ind1, ind2)

                toolbox.mutate(ind1, indpb=self.mutpb)
                toolbox.mutate(ind2, indpb=self.mutpb)
                del ind1.fitness.values, ind2.fitness.values

            # Evaluate the individuals with an invalid fitness
            print("##### Evaluation starts")
            start_time = time.time()

            ## fitness value 모음
            fit_list = []

            ## 가장 최소 fitnesss value1,2 구하기
            local_min_fit1 = float('inf')
            local_min_fit2 = float('inf')
            local_min_index = [None, None]

            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                fit_list.append(fit)

                if fit[0] < local_min_fit1:
                    local_min_fit1 = fit[0]
                    local_min_index[0] = ind

                if fit[1] < local_min_fit2:
                    local_min_fit2 = fit[1]
                    local_min_index[1] = ind

            ## index ckpt download

            ## log 기록
            train_log[gen] = fit_list
            self.train_log = train_log

            eval_time_for_one_generation = time.time() - start_time
            print("##### Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)

            # Select the next generation population
            pop = toolbox.select(pop + offspring, POP_SIZE)

            gen_time = time.time() - start_gen
            print('##### [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')

            record = stats.compile(pop)
            logbook.record(gen=gen, evals=len(invalid_ind), **record,
                           evals_time=eval_time_for_one_generation, gen_time=gen_time)

            logging.info('Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (
            gen, self.ngen, len(invalid_ind), eval_time_for_one_generation, gen_time))
            print(logbook.stream)

    ## Save Check point

    ## Save Log
    def save_log(self):

        log = self.log

        ## 필요한 log 추후 정리하여 추가
        log['train_log'] = self.train_log

        with open(self.train_log_file_name, 'w', encoding='utf-8') as make_file:
            json.dump(log, make_file, ensure_ascii=False, indent='\t')




### test 항목
* Train 되는지 - 됨, 다른 데이터셋은 아직 안됨
* log data 저장 되는지 떨꾸어지는지 - 됨
* 데이터셋 바뀌는지

In [3]:
temp = rwns_train("default.json")


{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 32,
                'data': 'CIFAR10',
                'epochs': 1,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'BASIC',
 'NGEN': 1,
 'NUM_GRAPH': 100,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}


In [4]:
temp.create_toolbox()

Start to make random graph pool...
Finished


In [5]:
temp.train()

Start to make random graph pool...
Finished


C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Start to make random graph pool...
Finished
Initialion starts ...
Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/1563]	Time 7.665 (7.665)	Loss 7.0445 (7.0445)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
	 - Epoch: [0][100/1563]	Time 0.427 (0.442)	Loss 2.1598 (2.5642)	Prec@1 21.875 (15.347)	Prec@5 71.875 (62.686)
	 - Epoch: [0][200/1563]	Time 0.336 (0.403)	Loss 2.1431 (2.3396)	Prec@1 21.875 (18.548)	Prec@5 65.625 (69.201)
	 - Epoch: [0][300/1563]	Time 0.356 (0.387)	Loss 1.9766 (2.2208)	Prec@1 34.375 (21.003)	Prec@5 81.250 (72.913)
	 - Epoch: [0][400/1563]	Time 0.357 (0.380)	Loss 1.7303 (2.1449)	Prec@1 21.875 (23.005)	Prec@5 96.875 (75.296)
	 - Epoch: [0][500/1563]	Time 0.339 (0.374)	Loss 1.3701 (2.0811)	Prec@1 59.375 (25.200)	Prec@5 96.875 (77.146)
	 - Epoch: [0][600/1563]	Time 0.358 (0.373)	Loss 1.9383 (2.0312)	Prec@1 18.750 (26.825)	Prec@5 87.500 (78.567)
	 - Epoch: [0][700/1563]	Time 0.370 (0.371)	Loss 1.7446 (1.9875)	Prec@1 28.125 (28.254)

In [6]:
temp.train_log

OrderedDict([(0,
              [(-50.94, 824595776.0),
               (-49.71, 934919296.0),
               (-44.92, 1071135616.0),
               (-48.27, 887918528.0)])])

In [7]:
temp.save_log()

### Generation 늘리기

In [16]:
long = rwns_train("default_long.json")
long.create_toolbox()
long.train()
long.save_log()

{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 32,
                'data': 'CIFAR10',
                'epochs': 1,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'BASIC_long',
 'NGEN': 3,
 'NUM_GRAPH': 100,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}
Start to make random graph pool...
Finished


C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Initialion starts ...
Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/1563]	Time 6.890 (6.890)	Loss 6.8758 (6.8758)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
	 - Epoch: [0][100/1563]	Time 0.474 (0.547)	Loss 2.3966 (2.6595)	Prec@1 9.375 (15.347)	Prec@5 65.625 (62.593)
	 - Epoch: [0][200/1563]	Time 0.473 (0.513)	Loss 2.0805 (2.4346)	Prec@1 9.375 (17.024)	Prec@5 71.875 (66.076)
	 - Epoch: [0][300/1563]	Time 0.477 (0.505)	Loss 2.1221 (2.3239)	Prec@1 9.375 (18.480)	Prec@5 81.250 (69.020)
	 - Epoch: [0][400/1563]	Time 0.485 (0.501)	Loss 1.9542 (2.2589)	Prec@1 25.000 (19.483)	Prec@5 93.750 (71.041)
	 - Epoch: [0][500/1563]	Time 0.472 (0.498)	Loss 2.1140 (2.2038)	Prec@1 21.875 (21.020)	Prec@5 78.125 (72.736)
	 - Epoch: [0][600/1563]	Time 0.483 (0.496)	Loss 1.6835 (2.1577)	Prec@1 31.250 (22.322)	Prec@5 90.625 (74.350)
	 - Epoch: [0][700/1563]	Time 0.474 (0.495)	Lo

##### [gen_time: 3768.986s] 1 th generation is finished.
1  	4    	[-5.0940000e+01  8.8614528e+08]  	[-4.60700000e+01  9.97494272e+08]	3768.98   	3768.99 
##### 2 th generation starts at 2019-10-11 16:40:14
##### Evaluation starts
Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/1563]	Time 6.879 (6.879)	Loss 6.9056 (6.9056)	Prec@1 0.000 (0.000)	Prec@5 3.125 (3.125)
	 - Epoch: [0][100/1563]	Time 0.504 (0.567)	Loss 1.9561 (2.4883)	Prec@1 28.125 (16.955)	Prec@5 81.250 (66.646)
	 - Epoch: [0][200/1563]	Time 0.538 (0.532)	Loss 1.6619 (2.2715)	Prec@1 40.625 (20.165)	Prec@5 90.625 (72.248)
	 - Epoch: [0][300/1563]	Time 0.513 (0.525)	Loss 2.0462 (2.1772)	Prec@1 12.500 (22.083)	Prec@5 84.375 (74.886)
	 - Epoch: [0][400/1563]	Time 0.480 (0.517)	Loss 1.7430 (2.1190)	Prec@1 46.875 (23.200)	Prec@5 84.375 (76.699)
	 - Epoch: [0][500/1563]	Time 0.475 (0.513)	Loss 1.7077 (2.0747)	Prec@1 43.750 (24.464)	Prec@5 87.500 (78.019)
	 - Epoch: [0][600/1563]	Time 0.4

#### 로그 성공

In [17]:
long.log

OrderedDict([('hp',
              {'data': 'CIFAR10',
               'lr_mode': 'cosine',
               'warmup_mode': 'linear',
               'base_lr': 0.1,
               'momentum': 0.9,
               'weight_decay': 5e-05,
               'print_freq': 100,
               'epochs': 1,
               'batch_size': 32,
               'workers': 4,
               'warmup_epochs': 0,
               'warmup_lr': 0.0,
               'targetlr': 0.0}),
             ('train_log',
              OrderedDict([(0,
                            [(-44.39, 946092416.0),
                             (-46.49, 915584768.0),
                             (-47.52, 1121832448.0),
                             (-49.53, 891870784.0)]),
                           (1,
                            [(-46.07, 886145280.0),
                             (-50.94, 997494272.0),
                             (-47.12, 1089700736.0),
                             (-48.75, 900245824.0)]),
                           (2,
 

### num_class 수정

In [22]:
num_class = rwns_train("num_class_test.json")
num_class.create_toolbox()
num_class.train()
num_class.save_log()

{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 32,
                'data': 'CIFAR10',
                'epochs': 1,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'num_classes_test',
 'NGEN': 1,
 'NUM_GRAPH': 100,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Initialion starts ...
Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/1563]	Time 6.833 (6.833)	Loss 6.8544 (6.8544)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000

### 데이터 셋 바꾸기


In [5]:
mnist_test = rwns_train("MNIST_TEST.json")
mnist_test.create_toolbox()
mnist_test.train()
mnist_test.save_log()

{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 32,
                'data': 'MNIST',
                'epochs': 1,
                'input_dim': 1,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'MNIST_TEST',
 'NGEN': 1,
 'NUM_GRAPH': 100,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Initialion starts ...
	 - Epoch: [0][0/1875]	Time 1.182 (1.182)	Loss 2.2980 (2.2980)	Prec@1 9.375 (9.375)	Prec@5 59.375 (59.375)
	 - Epoch: [0][100/1875]	Time 0.475 (0.511)	Loss

KeyboardInterrupt: 

123


### Model Save

In [7]:
import os
import sys
import os
import logging
from easydict import EasyDict
import numpy as np
import random
import time
import datetime
from deap import tools
from collections import OrderedDict
from pprint import pprint
import json
import torch

sys.path.insert(0, '../')
from utils_kyy.utils_graph import make_random_graph
from utils_kyy.create_toolbox import create_toolbox_for_NSGA_RWNN
from utils_kyy.create_toolbox import evaluate


class rwns_train:
    def __init__(self, json_file):
        self.root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        #self.root = os.getcwd()
        self.param_dir = os.path.join(self.root + '/parameters/', json_file)
        f = open(self.param_dir)
        params = json.load(f)
        pprint(params)
        self.name = params['NAME']
        self.log_dir = os.path.join(self.root, 'log')
        self.model_dir = os.path.join(self.root +'/saved_models/', self.name)
        
        
        # create directory
        if not (os.path.isdir(self.model_dir)):
            os.makedirs(self.model_dir)
        if not (os.path.isdir(self.log_dir)):
            os.makedirs(self.log_dir)


        ## toolbox params
        self.args_train = EasyDict(params['ARGS_TRAIN'])
        self.data_path = params['DATA_PATH']
        self.run_code = params['RUN_CODE']
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '/'
        self.num_graph = params['NUM_GRAPH']
        self.toolbox = None

        ## GA params
        self.pop_size = params['POP_SIZE']
        self.ngen = params['NGEN']
        self.cxpb = params['CXPB']
        self.mutpb = params['MUTPB']

        ## logs
        log = OrderedDict()

        log['hp'] = self.args_train
        self.log = log
        self.train_log = None 
        
        
        
    ## tool box and make graph
    def create_toolbox(self):
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '_' + self.name + '/'

        if not os.path.exists(self.stage_pool_path): os.makedirs(self.stage_pool_path)
        if not os.path.isdir(self.log_path): os.makedirs(self.log_path)
        self.log_file_name = self.log_path + 'logging.log'
        self.train_log_file_name = self.log_path + 'train_logging.log'

        logging.basicConfig(filename=self.log_file_name, level=logging.INFO)
        logging.info('Start to write log.')

        # num_graph = 100
        make_random_graph(self.num_graph, self.stage_pool_path)

        return create_toolbox_for_NSGA_RWNN(self.num_graph, self.args_train, self.stage_pool_path, self.data_path,
                                            self.log_file_name)

    ## Train

    def train(self):

                
        ## Parameters
        self.create_toolbox()
        POP_SIZE = self.pop_size

        ## train log
        train_log = OrderedDict()

        if self.toolbox is None:
            self.toolbox = self.create_toolbox()

        toolbox = self.toolbox

        # log에 기록할 stats
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "min", "max", "evals_time", "gen_time"

        # population 생성.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialion starts ...")
        logging.info("Initialion starts at " + now_str)
        init_start_time = time.time()

        pop = toolbox.population(n=POP_SIZE)

        ## fitness, model list
        fit_list = []
        model_list = []
        
        local_min_fit1 = float('inf')
        local_min_fit2 = float('inf')
        local_min_index = [None, None]
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        #fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)  # .evaluate는 tuple을 반환. 따라서 fitnesses는 튜플을 원소로 가지는 list
        
        for idx, ind in enumerate(invalid_ind):
            fitness, ind_model = evaluate(ind,args_train=self.args_train, stage_pool_path=self.stage_pool_path,data_path=self.data_path ,log_file_name=self.log_file_name)
            ind.fitness.values = fitness
            fit_list.append(fitness)
            model_list.append(ind_model)
            
            if fitness[0] < local_min_fit1:
                local_min_fit1 = fitness[0]
                local_min_index[0] = idx

            if fitness[1] < local_min_fit2:
                local_min_fit2 = fitness[1]
                local_min_index[1] = idx

        ## index ckpt download
        print("#### Saving Model", local_min_index)
        self.save_model(model=model_list[local_min_index[0]],ngen=0,subname=str(0)+'_'+'acc')                          
        self.save_model(model=model_list[local_min_index[1]],ngen=0,subname=str(0)+'_'+'flops')                          

            
        ## log 기록
        train_log[0] = fit_list
        self.train_log = train_log

        # This is just to assign the crowding distance to the individuals
        # no actual selection is done
        pop = toolbox.select(pop, len(pop))

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(invalid_ind), **record)
        print(logbook.stream)

        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialization is finished at", now_str)
        logging.info("Initialion is finished at " + now_str)

        init_time = time.time() - init_start_time
        logging.info("Initialion time = " + str(init_time) + "s")

        print()

        # Begin the generational process
        for gen in range(1, self.ngen):
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d %H:%M:%S')
            print("#####", gen, "th generation starts at", now_str)
            logging.info(str(gen) + "th generation starts at" + now_str)

            start_gen = time.time()
            # Vary the population
            offspring = tools.selTournamentDCD(pop, len(pop))
            offspring = [toolbox.clone(ind) for ind in offspring]

            for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
                if random.random() <= self.cxpb:
                    toolbox.mate(ind1, ind2)

                toolbox.mutate(ind1, indpb=self.mutpb)
                toolbox.mutate(ind2, indpb=self.mutpb)
                del ind1.fitness.values, ind2.fitness.values

            # Evaluate the individuals with an invalid fitness
            print("##### Evaluation starts")
            start_time = time.time()

            ## fitness value : accuracy ,flops 모음
            fit_list = []
            model_listist = []

            ## 가장 최소 1,2 구하기
            local_min_fit1 = float('inf')
            local_min_fit2 = float('inf')
            local_min_index = [None, None]

            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            
            for idx,ind in enumerate(invalid_ind):
                fitness, ind_model = evaluate(ind,args_train=self.args_train, stage_pool_path=self.stage_pool_path,data_path=self.data_path ,log_file_name=self.log_file_name)
                ind.fitness.values = fitness
                fit_list.append(fitness)
                model_list.append(ind_model)

                if fitness[0] < local_min_fit1:
                    local_min_fit1 = fitness[0]
                    local_min_index[0] = idx

                if fitness[1] < local_min_fit2:
                    local_min_fit2 = fitness[1]
                    local_min_index[1] = idx


            ## index ckpt download
            print("#### Saving Model", local_min_index)
            self.save_model(model=model_list[local_min_index[0]],ngen=gen,subname=str(gen)+'_'+'acc')                          
            self.save_model(model=model_list[local_min_index[1]],ngen=gen,subname=str(gen)+'_'+'flops')                          

            ## log 기록
            train_log[gen] = fit_list
            self.train_log = train_log

            eval_time_for_one_generation = time.time() - start_time
            print("##### Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)

            # Select the next generation population
            pop = toolbox.select(pop + offspring, POP_SIZE)

            gen_time = time.time() - start_gen
            print('##### [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')

            record = stats.compile(pop)
            logbook.record(gen=gen, evals=len(invalid_ind), **record,
                           evals_time=eval_time_for_one_generation, gen_time=gen_time)

            logging.info('Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (
            gen, self.ngen, len(invalid_ind), eval_time_for_one_generation, gen_time))
            print(logbook.stream)

    ## Save Check point

    ## Save Log
    def save_log(self):

        log = self.log

        ## 필요한 log 추후 정리하여 추l가
        log['train_log'] = self.train_log

        with open(self.train_log_file_name, 'w', encoding='utf-8') as make_file:
            json.dump(log, make_file, ensure_ascii=False, indent='\t')

    ## Save Model
    def save_model(self, model,ngen, subname):
        
        model_fname = self.name + '_' + str(ngen) + '_' + subname
        model_path = os.path.join(self.model_dir, model_fname)
        print("Saving Model",model_path)
        torch.save(model,model_path)


In [9]:
num_class = rwns_train("num_class_test.json")
num_class.create_toolbox()
num_class.train()
num_class.save_log()

{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 32,
                'data': 'CIFAR10',
                'epochs': 1,
                'input_dim': 3,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'num_classes_test',
 'NGEN': 2,
 'NUM_GRAPH': 100,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Initialion starts ...
Files already downloaded and verified
Files already downloaded and verified
	 - Epoch: [0][0/1563]	Time 6.926 (6.926)	Loss 2.2957 (2.2957)	Prec@1 1

### 모델 저장 성공
* evaluate은 밖에서 하는 걸로

In [32]:
a

inf

In [ ]:
run_code = 'test_kyy_CIFAR10_time_check'
stage_pool_path = '../graph_pool' + '/' + run_code + '/'
log_path = '../logs/' + run_code + '/'

data_path = 'D:/data/cifar10/'
toolbox = create_toolbox_for_NSGA_RWNN(num_graph, args_train, stage_pool_path, data_path ,log_file_name)


In [ ]:
POP_SIZE = 2    # population size
NGEN = 2    # number of Generation
CXPB = 0.5    # crossover probability 
MUTPB = 0.5    # mutation probability


In [14]:
## Log book tutorial
import numpy
import random

from deap import algorithms, base
from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
pop = toolbox.population(n=100)


C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


AttributeError: 'list' object has no attribute 'fitness'

In [16]:
stat

NameError: name 'stat' is not defined